# SIEM and SBM Tests

In [22]:
import graspy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import dask
from dask.distributed import Client, progress
import dask.dataframe as ddf
#uncomment when placed into model folder not doc
#from .base import BaseGraphEstimator, _calculate_p

from scipy.stats import bernoulli, mannwhitneyu, fisher_exact
from scipy import stats
from scipy.special import expit
import pandas as pd 
import mizani as miz

from graspy.simulations import sbm, er_np, er_nm
from graspy.plot import heatmap

import statsmodels.api as sm
import statsmodels.formula.api as smf

from graspy.models.base import BaseGraphEstimator 
from graspy.utils.utils import (
    augment_diagonal,
    cartprod,
    import_graph,
    is_unweighted,
    remove_loops,
    symmetrize,
)
from graspy.simulations import siem
import sys
import plotnine as p9
from dfply import *
import pickle as pkl

In [6]:
def modular_edges(n):
    """
    A function for generating modular sbm edge communities.
    """
    m = int(n/2)
    edge_comm = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if ( (i<m) & (j<m)) or ( (i>=m ) & (j>=m) ):
                edge_comm[i,j] = 1
            else:
                edge_comm[i,j] = 2   
    return edge_comm

def nuis_edges(n):
    """
    A function for generating doubly modular sbm.
    """
    m = int(n/2)
    m4 = int(7*n/8)
    m3 = int(5*n/8)
    m2 = int(3*n/8)
    m1 = int(1*n/8)
    edge_comm = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if ( (i<m) & (j<m)) or ( (i>=m ) & (j>=m) ):
                edge_comm[i,j] = 1
            elif (((i >= m3) & (i <= m4)) & ((j >= m1) & (j <= m2))) or 
            (((i >= m1) & (i <= m2)) & ((j >= m3) & (j <= m4))):
                edge_comm[i,j] = 3
            else:
                edge_comm[i,j]=2
    return edge_comm
    

def diag_edges(n):
    """
    A function for generating diagonal SIEM edge communities.
    """
    m = int(n/2)
    edge_comm = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if (i == j + m) or (j == i + m):
                edge_comm[i,j] = 1
            else:
                edge_comm[i,j] = 2
    return edge_comm

            
def modular_sbm(n, effect_size=0, weighted=False):
    """
    A function to generate edge assignments for a modular sbm.
    """
    edge_comm = modular_edges(n)
    if (weighted == False):
        mod = siem(n, [0.5 + effect_size/2, 0.5 - effect_size/2],
                   edge_comm=edge_comm, directed = True, loops = True,
                   return_labels=True)
    else:
        mod = siem(n, [1, 1],
                   edge_comm=edge_comm, directed = True, loops = True,
                   return_labels=True, wt=np.random.normal,
                    wtargs=[{"loc": 0.5 + effect_size/2, "scale": .5},
                            {"loc": 0.5 - effect_size/2, "scale": .5}])
    mod = list(mod)
    return tuple(mod)

def outlier_sbm(n, effect_size=0, outlier=.25, outlier_sc=3):
    """
    A function to generate edge assignments for a modular sbm.
    """
    edge_comm = modular_edges(n)

    mod = siem(n, [1, 1],
               edge_comm=edge_comm, directed = True, loops = True,
               return_labels=True, wt=np.random.normal,
                wtargs=[{"loc": 0.5 + effect_size/2, "scale": .5},
                        {"loc": 0.5 - effect_size/2, "scale": .5}])
    mod = list(mod)
    mod[0] = mod[0] + (np.multiply(np.random.binomial(n=1, p=outlier, size=n**2),
        np.random.normal(loc = 0, scale=outlier_sc, size=n**2))).reshape((n,n))
    return tuple(mod)

def skew_modular_sbm(n, effect_size=0):
    """
    A function to generate edge weights for a modular sbm,
    with skew in the weight distribution.
    """
    C = modular_edges(n)
    A = np.zeros((n,n))
    A[C == 1] = np.random.beta(5, 5 + effect_size, size=(C == 1).sum())
    A[C == 2] = np.random.beta(5 - effect_size, 5, size=(C == 2).sum())
    return (A, C)
        
def diag_siem(n, effect_size=0, weighted=False):
    """
    A function to generate edge assignments for a diagonal siem.
    """
    edge_comm = diag_edges(n)
    if (weighted == False):
        mod = siem(n, [0.5 + effect_size/2, 0.5 - effect_size/2],
                   edge_comm=edge_comm, directed = True, loops = True,
                   return_labels=True)
    else:
        mod = siem(n, [1, 1],
                   edge_comm=edge_comm, directed = True, loops = True,
                   return_labels=True, wt=np.random.normal,
                    wtargs=[{"loc": 0.5 + effect_size/2, "scale": .5},
                            {"loc": 0.5 - effect_size/2, "scale": .5}])
    mod = list(mod)
    return tuple(mod)

def vcorr_sbm(n, effect_size=0):
    """
    A function to generate a vertex-correlated SBM.
    """
    C = modular_edges(n).flatten()
    X = np.zeros((n, n))

    Vj = np.tile(range(0, n), (n,1)).flatten()
    Vi = np.tile(range(0, n), (n, 1)).transpose().flatten()
    # the portion of the design matrix for the vertex assignments
    # n elements (1 parameter per vertex) and n^2 entries (1 vector per edge)
    X_v=np.zeros((n**2, n))
    for i in range(0, len(Vi)):
        X_v[i, Vi[i]] = 1
        X_v[i, Vj[i]] = 1
    # create design matrix with an intercept, an offset for community 1,
    # and succeeding columns are the incident vertex ids
    X = np.hstack((np.ones((n**2))[:,None], (C == 1)[:,None], X_v))
    # coefficients are each vertex is standard normal for the "amplifying factor"
    B = np.array(np.vstack((np.array([-effect_size/2, effect_size/2])[:,None],
                            np.random.uniform(low=-5, high=5, size=n)[:,None])))
    # systematic component of the GLM
    Sys_comp = X.dot(B)
    # convert to probabilities
    P = 1/(1 + np.exp(Sys_comp))
    # sample the graph
    Edges = bernoulli.rvs(P)
    # put it into adjacency form
    return (Edges.reshape((n, n)), C.reshape((n,n)))

def nuisance_sbm(n, effect_size=0, nuis=.7):
    """
    A SBM, with a nuisance community.
    """
    C = nuis_edges(n)
    eff = [0.5 + effect_size/2, 0.5 - effect_size/2, nuis]
    A = np.zeros((n,n))
    for k in range(1, 4):
        A[C == k] = np.random.binomial(1, eff[k-1], size=(C == k).sum())
    return A, C

def corr_sbm(n, effect_size=0, corr=.5):
    """
    A SBM with correlated between-community relationship.
    """
    # start with P=.5
    P = 0.5*np.ones((n,n))
    # in the upper left and bottom right blocks, make .5 + effect_size/2
    P[1:(n/2), 1:(n/2)] = P[1:(n/2), 1:(n/2)] + effect_size/2
    P[((n/2)+1):n, ((n/2)+1):n] = P[((n/2)+1):n, ((n/2)+1):n] + effect_size/2
    P[1:(n/2), ((n/2)+1):n] = P[1:(n/2), ((n/2)+1):n] - effect_size/2
    P[((n/2)+1):n, 1:(n/2)] = P[((n/2)+1):n, 1:(n/2)] - effect_size/2
    
        
def fishers_exact(G, C, alternative='neq'):
    """
    G is a binary graph, C is community assignments.
    """
    if alternative == 'neq':
        alternative = "two-sided"
    T = np.zeros((2,2))
    for k in range(0, 2):
        T[0,k] = (G[C == (k+1)]).sum()
        T[1,k] = len(G[C == (k + 1)]) - T[0,k]
    (T, pval) = fisher_exact(T, alternative=alternative)
    return(pval)
    
def lrt(G, C):
    G_vec = G.flatten()
    C_vec = C.flatten()
    model = smf.glm(formula='Edges~Community',
                    data=pd.DataFrame({'Edges': G_vec, 'Community': C_vec}),
                    family=sm.families.Binomial()).fit()
    model_null = smf.glm(formula='Edges~1',
                    data=pd.DataFrame({'Edges': G_vec, 'Community': C_vec}),
                    family=sm.families.Binomial()).fit()
    dof = model_null.df_resid - model.df_resid
    lrs = 2*(model.llf - model_null.llf)
    lr_pval=stats.chi2.sf(lrs, df=dof)
    return(lr_pval)

def welch_t(G, C):
    A = G[C == 1]
    B = G[C == 2]
    return(stats.ttest_ind(A, B, equal_var=False)[1])

def mww(G, C):
    A = G[C == 1]
    B = G[C == 2]
    return(stats.mannwhitneyu(A, B, alternative='two-sided')[1])

def uncorr_perm(G, C, nperm=200):
    G_vec = G.flatten()
    C_vec = C.flatten()
    if len(np.unique(G_vec)) > 2:
        wt_fn = np.median
    else:
        wt_fn = np.mean
    Ta = np.abs(wt_fn(G_vec[C_vec == 1]) - wt_fn(G_vec[C_vec == 2]))
    T_null = np.zeros((nperm))
    for i in range(0, nperm):
        C_null = np.random.choice(C_vec, size=len(C_vec), replace=False)
        T_null[i] = wt_fn(G_vec[C_null == 1]) - wt_fn(G_vec[C_null == 2])
    T_null = np.abs(T_null)
    return((sum(T_null > Ta)+1)/(nperm + 1))

def corr_lrt(G, C):
    G_vec = G.flatten()
    Vi = np.tile(range(0, C.shape[0]), (C.shape[0],1)).flatten()
    Vj = np.tile(range(0, C.shape[0]), (C.shape[0], 1)).transpose().flatten()
    C_vec = C.flatten()
    edge_df = pd.DataFrame({'Edges': G_vec, 'Community': C_vec,
                            'Vi': Vi, 'Vj': Vj})
    model = smf.glm(formula='Edges~Community + Vi + Vj',
                    data=edge_df, family=sm.families.Binomial()).fit()
    model_null = smf.glm(formula='Edges~Vi + Vj',
                    data=edge_df, family=sm.families.Binomial()).fit()
    dof = model_null.df_resid - model.df_resid
    lrs = 2*(model.llf - model_null.llf)
    lr_pval=stats.chi2.sf(lrs, df=dof)
    return(lr_pval)

def nlrt(G):
    n = G.shape[0]
    G_vec = G.flatten()
    C = nuis_edges(n)
    C_vec = C.flatten()
    edge_df = pd.DataFrame({'Edges': G_vec, 'Community': C_vec})
    edge_df["Community"] = edge_df["Community"].astype("category")
    model = smf.glm(formula='Edges~Community',
                    data=edge_df, family=sm.families.Binomial()).fit()
    model_null = smf.glm(formula='Edges~(Community == 2)',
                    data=edge_df, family=sm.families.Binomial()).fit()
    dof = model_null.df_resid - model.df_resid
    lrs = 2*(model.llf - model_null.llf)
    lr_pval=stats.chi2.sf(lrs, df=dof)
    return(lr_pval)

def gather_matrix(mtx, weighted="", gr_type=""):
    """
    Converts a matrix to a pandas dataframe
    """
    pd_mtx = pd.DataFrame(mtx)
    pd_mtx.columns = np.arange(1, mtx.shape[0] + 1, 1)
    pd_mtx >>= gather('Column', 'Weight', add_id=True)
    pd_mtx=pd_mtx.rename(columns={"_ID": "Row"})
    pd_mtx["Weighting"]=weighted
    pd_mtx["Method"] = gr_type
    pd_mtx["Row"] = pd_mtx["Row"] + 1
    return(pd_mtx)

In [7]:
# the color for each test statistic
test_stat_cols = {'FET': 'red',
                  'LRT': 'teal',
                  'MWW': 'blue',
                  'PT': 'green',
                  'WT': 'orange',
                  'rLRT': 'purple'}

# an array tracking the test statistics themselves
test_stats = {'FET': fishers_exact,
              'LRT': lrt,
              'MWW': mww,
              'PT': uncorr_perm,
              'WT': welch_t,
              'rLRT': corr_lrt}

# an array for when to use different test statistics
test_stat_use = {'FET': ['Unweighted'],
                'LRT': ['Unweighted'],
                'MWW': ['Unweighted', 'Weighted'],
                'PT': ['Unweighted', 'Weighted'],
                'WT': ['Unweighted', 'Weighted'],
                'rLRT': ['Unweighted'],
                'nLRT': ['Unweighted']}

simulations = {
    'Unweighted': {'SIEM': {'fn': diag_siem, 'eff_sz': np.linspace(0, .4, 10), 'kwarg': {"weighted": False}},
                   'SBM': {'fn': modular_sbm, 'eff_sz': np.linspace(0, .2, 10), 'kwarg': {"weighted": False}},
                   'rSBM': {'fn': vcorr_sbm, 'eff_sz': np.linspace(0, 3, 10), 'kwarg': None},
                   'nSBM': {'fn': nuisance_sbm, 'eff_sz': np.linspace(0, .5, 10), 'kwarg': None}},
    'Weighted': {'SIEM': {'fn': diag_siem, 'eff_sz': np.linspace(0, .3, 10), 'kwarg': {"weighted": True}}, 
                 'SBM': {'fn': modular_sbm, 'eff_sz': np.linspace(0, .2, 10), 'kwarg': {"weighted": True}},
                 'SSBM': {'fn': skew_modular_sbm, 'eff_sz': np.linspace(0, 2, 10), 'kwarg': None},
                 'oSBM': {'fn': outlier_sbm, 'eff_sz': np.linspace(0, .2, 10), 'kwarg': {"outlier": .3}}}
}

nrep = 100
ncores = 6
nvertices = np.round(np.linspace(20, 100, 10))

In [8]:
client = Client(threads_per_worker=1, n_workers=ncores)
client

Client Scheduler: tcp://127.0.0.1:33969 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 6 Memory: 33.51 GB


# Simulations Driver

In [9]:
exps = []
for weighting, sims in simulations.items():
    for sim_name, sim in sims.items():
        for es in sim["eff_sz"]:
            for n in nvertices:
                for i in range(nrep):
                    exps.append([sim_name, weighting, n, es, i])
sim_exps = pd.DataFrame(exps, columns=["Simulation", "Weighting", "n", "Effect_Size", "i"])
sim_exps["n"] = sim_exps["n"].astype(int)
print(sim_exps.head())
print(sim_exps.shape)

  Simulation   Weighting   n  Effect_Size  i
0       SIEM  Unweighted  20          0.0  0
1       SIEM  Unweighted  20          0.0  1
2       SIEM  Unweighted  20          0.0  2
3       SIEM  Unweighted  20          0.0  3
4       SIEM  Unweighted  20          0.0  4
(80000, 5)


In [10]:
def run_exp(row):
    # generate simulation using the dictionary defined above
    sim_dict = simulations[row["Weighting"]][row["Simulation"]]
    if sim_dict["kwarg"] is not None:
        (A, C) = sim_dict["fn"](row["n"], row["Effect_Size"], **sim_dict["kwarg"])
    else:
        (A, C) = sim_dict["fn"](row["n"], row["Effect_Size"])
    # run sim on all test statistics
    p_vals = []
    stat_names = []
    for test_name, test in test_stats.items():
        # check if statistic is appropriate for the setting
        if row["Weighting"] in test_stat_use[test_name]:
            try:
                pval = test(A, C)
                p_vals.append(pval)
                stat_names.append(test_name)
            except:
                ex_str = "Failed Test: {}, Simulation: {}, n: {}, Effect Size: {}"
                print(ex_str.format(test_name, row["Simulation"], row["n"], row["Effect_Size"]))
                p_vals.append(float("NaN"))
        else:
            p_vals.append(float("NaN"))
    return tuple([row["Simulation"], row["Weighting"], row["n"],
                 row["Effect_Size"], row["i"], *p_vals])

In [7]:
sim_exps = ddf.from_pandas(sim_exps, npartitions=ncores*1.5)
sim_results = sim_exps.apply(lambda x: run_exp(x), axis=1, result_type='expand',
                             meta={0: str, 1: str, 2: int, 3: float, 4: int,
                                   5: float, 6: float, 7: float, 8: float, 9: float, 10: float})

In [8]:
sim_results = sim_results.compute(scheduler="multiprocessing")
sim_results = sim_results.rename(columns={0: "Simulation", 1: "Weighting", 2: "n", 3: "Effect_Size",
                                          4: "i", 5: "FET", 6: "LRT", 7: "MWW", 8: "PT",
                                          9: "WT", 10: "rLRT"})
sim_results.to_pickle('./data/singlegraph_results.pkl')

In [18]:
sim_results = pd.read_pickle('./data/singlegraph_results.pkl')
sim_results >>= gather('Test', 'p_val', ['FET', 'LRT', 'MWW', 'PT', 'WT', 'rLRT'])
sim_results.head()

,Simulation,Weighting,n,Effect_Size,i,Test,p_val
0,SIEM,Unweighted,20,0.0,0,FET,0.819421
1,SIEM,Unweighted,20,0.0,1,FET,0.169889
2,SIEM,Unweighted,20,0.0,2,FET,0.250902
3,SIEM,Unweighted,20,0.0,3,FET,1.000000
4,SIEM,Unweighted,20,0.0,4,FET,0.496044


In [45]:
# threshold and compute power
@make_symbolic
def purge_nan(series):
    return np.invert(np.isnan(series))

@make_symbolic
def std_err(series):
    return np.std(series)/len(series)
alpha = .05
power_results = sim_results >> mask(purge_nan(X.p_val)) >> group_by(
    "Simulation", "Weighting", "n", "Effect_Size", "Test"
    ) >> mutate(outcome=(X.p_val < alpha).astype(float)
    ) >> summarize(power=X.outcome.mean(), power_stderr=std_err(X.outcome)
    ) >> group_by(
        "Simulation", "Weighting", "n"
    ) >> mutate(  # normalize the effect size so they are all on the same scale
        Effect_Size=np.around((X.Effect_Size - np.min(X.Effect_Size))/(np.max(X.Effect_Size)-np.min(X.Effect_Size)),
                             decimals=3)
    )
power_results["n"] = power_results["n"].astype('category')
power_results["n"].cat.reorder_categories(nvertices.astype(int), inplace=True)
power_results["Effect_Size"] = power_results["Effect_Size"].astype('category')
power_results["Effect_Size"].cat.reorder_categories(
    np.sort(np.unique(power_results["Effect_Size"])), inplace=True)
power_results.head(10)

,Test,Effect_Size,n,Weighting,Simulation,power,power_stderr
0,FET,0.000,20,Unweighted,SBM,0.07,0.002551
1,LRT,0.000,20,Unweighted,SBM,0.10,0.003000
2,MWW,0.000,20,Unweighted,SBM,0.10,0.003000
3,PT,0.000,20,Unweighted,SBM,0.09,0.002862
4,WT,0.000,20,Unweighted,SBM,0.10,0.003000
5,rLRT,0.000,20,Unweighted,SBM,0.10,0.003000
6,FET,0.111,20,Unweighted,SBM,0.05,0.002179
7,LRT,0.111,20,Unweighted,SBM,0.06,0.002375
8,MWW,0.111,20,Unweighted,SBM,0.06,0.002375
9,PT,0.111,20,Unweighted,SBM,0.08,0.002713


# Unweighted Data Saving

In [46]:
uw_results = power_results >> mask(X.Weighting == "Unweighted")

gr_dat = pd.concat([gather_matrix(modular_sbm(50, effect_size=.5, weighted=False)[0], gr_type="SBM"),
           gather_matrix(diag_siem(50, effect_size=.8, weighted=False)[0], gr_type="SIEM"),
           gather_matrix(vcorr_sbm(50, effect_size=8)[0], gr_type="rSBM")])

power_fixed_dat = uw_results >> mask(X.n == 56)

uw_dat = {"power_map": uw_results, "simulations": gr_dat, "fixedn": power_fixed_dat}

for (key, value) in uw_dat.items():
    with open('./data/uwt_{}.csv'.format(key), 'w') as f:
        value.to_csv(f)

## Power Plots

,Test,Effect_Size,n,Weighting,Simulation,power,power_stderr
240,FET,0.0,56,Unweighted,SBM,0.06,0.002375
241,LRT,0.0,56,Unweighted,SBM,0.06,0.002375
242,MWW,0.0,56,Unweighted,SBM,0.06,0.002375
243,PT,0.0,56,Unweighted,SBM,0.06,0.002375
244,WT,0.0,56,Unweighted,SBM,0.06,0.002375
...,...,...,...,...,...,...,...
3295,LRT,1.0,56,Unweighted,rSBM,1.00,0.000000
3296,MWW,1.0,56,Unweighted,rSBM,1.00,0.000000
3297,PT,1.0,56,Unweighted,rSBM,1.00,0.000000
3298,WT,1.0,56,Unweighted,rSBM,1.00,0.000000


## Simulation Plots

In [ ]:
uw_sim_plt = (p9.ggplot(data=gr_dat, mapping=p9.aes(x="factor(Row)", y="factor(Column)", fill="factor(Weight)"))
 + p9.geom_tile()
 + p9.facet_grid("Method~", switch="y")
 + p9.xlab("Vertex 1")
 + p9.ylab("Vertex 2")
 + p9.scale_x_discrete(breaks=[1, 50])
 + p9.scale_y_reverse(breaks=[1, 50], expand=[0,])
 + p9.scale_fill_discrete(name="Edge")
 + p9.theme_bw()
 + p9.theme(figure_size=(1.5, 4.5))
 + p9.ggtitle("(I) Simulation Plot")
)

## Power (fixed $n$) plots

In [ ]:

uw_power_fixn_plt = (p9.ggplot(data=power_fixed_dat, mapping=p9.aes(x="Effect_Size", y="power", 
                                                color="factor(Test)", group="factor(Test)"))
 + p9.geom_line()
 + p9.facet_grid("Simulation~")
 + p9.xlab("Relative Effect Size")
 + p9.ylab("Power")
 + p9.scale_x_discrete(breaks=[0, 0.5, 1])
 + p9.scale_y_continuous(limits=[0, 1], breaks=[0, 1])
 + p9.scale_color_discrete(name="Test")
 + p9.geom_hline(mapping=p9.aes(yintercept=alpha))
 + p9.theme_bw()
 + p9.theme(figure_size=(1.5, 4.5), strip.text.y = p9.element_blank())
 + p9.ggtitle("(III) Power Plots, n=56")
)

# Weighted Plots

## Power Plots

In [48]:
wt_results = power_results >> mask(X.Weighting == "Weighted")

gr_dat = pd.concat([gather_matrix(modular_sbm(50, effect_size=.5, weighted=True)[0], gr_type="SBM"),
           gather_matrix(diag_siem(50, effect_size=.8, weighted=True)[0], gr_type="SIEM"),
           gather_matrix(skew_modular_sbm(50, effect_size=3)[0], gr_type="SSBM"),
           gather_matrix(outlier_sbm(50, effect_size=.8, outlier=.4, outlier_sc=1.5)[0], gr_type="oSBM")])
gr_dat >>= group_by("Method") >> mutate(Weight=(X.Weight - np.min(X.Weight))/(np.max(X.Weight) - np.min(X.Weight)))

power_fixed_dat = wt_results >> mask(X.n == 56)

wt_dat = {"power_map": wt_results, "simulations": gr_dat, "fixedn": power_fixed_dat}

for (key, value) in wt_dat.items():
    with open('./data/wt_{}.csv'.format(key), 'w') as f:
        value.to_csv(f)

In [ ]:
wt_sim_power_plt = (p9.ggplot(data=wt_results, mapping=p9.aes(x="n", y="Effect_Size", fill="power"))
 + p9.geom_tile()
 + p9.facet_grid("Simulation ~ Test")
 + p9.xlab("Number of Vertices")
 + p9.ylab("Effect Size")
 + p9.scale_fill_gradient(high="green", low="red", name="Power")
 + p9.scale_x_discrete(breaks=[20, 100])
 + p9.scale_y_discrete(breaks=[0, 1])
 + p9.theme_bw()
 + p9.theme(figure_size=(4.5, 6))
 + p9.ggtitle("(II) Weighted Power Plot")
)

## Simulation Plots

In [ ]:
gr_dat.head()

In [ ]:
(p9.ggplot(data=gr_dat, mapping=p9.aes(x="factor(Row)", y="factor(Column)", fill="Weight"))
 + p9.geom_tile()
 + p9.facet_grid("Method~")
 + p9.xlab("Vertex 1")
 + p9.ylab("Vertex 2")
 + p9.scale_x_discrete(breaks=[1, 50])
 + p9.scale_y_reverse(breaks=[1, 50])
 + p9.theme_bw()
 + p9.theme(figure_size=(1.5, 6))
 + p9.ggtitle("(I) Simulation Plot")
)

## Power (fixed $n$) plots

In [ ]:

wt_power_fixn_plt = (p9.ggplot(data=power_fixed_dat, mapping=p9.aes(x="Effect_Size", y="power", 
                                                color="factor(Test)", group="factor(Test)"))
 + p9.geom_line()
 + p9.facet_grid("Simulation~")
 + p9.xlab("Relative Effect Size")
 + p9.ylab("Power")
 + p9.scale_x_discrete(breaks=[0, 0.5, 1])
 + p9.scale_y_continuous(limits=[0, 1], breaks=[0, 1])
 + p9.scale_color_discrete(name="Test")
 + p9.geom_hline(mapping=p9.aes(yintercept=alpha))
 + p9.theme_bw()
 + p9.theme(figure_size=(1.5, 6))
 + p9.ggtitle("(III) Power Plots, n=56")
)